In [11]:
import nest_asyncio
nest_asyncio.apply()

In [12]:
import os
from dotenv import load_dotenv
import rich
from agents import Agent, Runner, GuardrailFunctionOutput, InputGuardrail
from pydantic import BaseModel
# import asyncio

In [13]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"
# print(api_key)

In [14]:
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

In [15]:
math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description = "Specialist agent for math questions",
    instructions = "You provide help with math problems. Explain your reasoning at each step and include examples.",
    model=MODEL
    
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description = "Specialist agent for historical questions",
    instructions = "You provide assistance with historical queries. Explain import events and context clearly.",
    model=MODEL
)

In [16]:
guardrail_agent = Agent(
    name="Guardrail Check",
    instructions = "Check if the user is asking about homework",
    output_type = HomeworkOutput,
    # model=MODEL
)

In [7]:
async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    rich.print(result)
    rich.print(final_output)
    return GuardrailFunctionOutput(
        output_info = final_output,
        tripwire_triggered = not final_output.is_homework
    )

Handoffs

In [8]:
triage_agent = Agent(
    name="Triage Agent",
    instructions = "You determine which agent to use based on the user's homework question.",
    handoffs = [math_tutor_agent, history_tutor_agent],
    input_guardrails= [
        InputGuardrail(guardrail_function=homework_guardrail)
    ],
    model=MODEL
)

Run the agent Orchestration

In [9]:
result = await Runner.run(
    triage_agent,
    "who was the first president of the united states?"
    # "Who won the first world war?"
)
print(result.final_output)

HomeworkOutput(
    is_homework=True,
    reasoning='The question is factual and common in educational contexts, typically appearing in history lessons 
or quizzes.'
)

The first president of the United States was George Washington. He took office on April 30, 1789, after being unanimously elected by the Electoral College. Washington is often referred to as the "Father of His Country" for his leadership during the American Revolutionary War and his role in drafting the U.S. Constitution. His presidency set many precedents for the office, including the tradition of a two-term limit, which was later formalized in the 22nd Amendment. Washington served two terms, leaving office in 1797, and is celebrated for his commitment to the principles of democracy and republicanism.


In [10]:
result = await Runner.run(
    triage_agent,
    "what is life"
)
print(result.final_output)

HomeworkOutput(
    is_homework=False,
    reasoning="The question 'what is life' is a philosophical or existential inquiry rather than a homework 
question. It often relates to discussions about the nature, purpose, and value of existence."
)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire